In [ ]:
# 데이콘 주식 공모전
# 팀 : 최규진
# 팀원 : 최규진, 심태율, 김홍표, 유진권

In [ ]:
#모듈 설치
!pip install kofia
!pip install pmdarima
!pip install --upgrade pip
!pip install scikit-learn
!pip install finance-datareader
!pip install pandas
!pip install numpy
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.0 MB/s eta 0:00:00


In [1]:
#라이브러리 임포트
from pykrx import stock
import pandas_datareader as pdr
from sklearn.model_selection import train_test_split
import FinanceDataReader as fdr
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pmdarima as pm
from pmdarima.arima import ndiffs
warnings.filterwarnings("ignore")


ModuleNotFoundError: ignored

In [ ]:
# -*- coding: utf-8 -*-
##############################################################################################################################################################

# 주가 시계열 분석

# 팀이름: 최규진

# 작성자: 최규진, 심태율, 김홍표, 유진권

##############################################################################################################################################################

#시드 세팅

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

##############################################################################################################################################################

# 트레인셋 불러오기

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 확인
file_path =  '/content/drive/MyDrive/데이콘 주식 공모전 아이티윌/'
train = pd.read_csv(file_path + 'train.csv')
train.shape


'''
아이티윌 컴 경로
file_path = 'C:/Users/itwill/Desktop/'
train = pd.read_csv(file_path + 'train.csv')
train.shape
'''


'''
#트레인과 테스트로 나누기 (8:2)
X_train, X_test = train_test_split(train, test_size=0.2, shuffle = False)
'''

# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return', 'rmse'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

##############################################################################################################################################################

start_date = datetime.strptime(str(train['일자'].min()), '%Y%m%d')
end_date = datetime.strptime(str(train['일자'].max()), '%Y%m%d')

# KOSPI 지수, USD/KRW 환율, 비트코인 데이터 가져오기
df_kospi = fdr.DataReader('KS11', start_date, end_date)
df_USD = fdr.DataReader('USD/KRW', start_date, end_date)
df_bitcoin = fdr.DataReader('BTC/KRW', start_date, end_date)
df_oil = fdr.DataReader('OIL',  start_date, end_date)
df_gold = fdr.DataReader('GOLD', start_date, end_date)
df_interest = fdr.DataReader('DGS', start_date, end_date)

df_bond = pd.read_csv(file_path + 'bond.csv')
df_bond['일자'] = pd.to_datetime(df_bond['일자'])
df_bond_sorted = df_bond[['일자', '국채_거래대금']].sort_values('일자')
df_bond_sorted.set_index('일자', inplace=True)
df_bond_sorted['국채_거래대금'] = np.log(df_bond_sorted['국채_거래대금'].str.replace(',', '').astype(float))

df_MO = pd.read_csv(file_path + 'MO.csv')
df_MO['일자'] = pd.to_datetime(df_MO['일자'])
df_MO_sorted = df_MO[['일자', 'MO']].sort_values('일자')
df_MO_sorted.set_index('일자', inplace=True)

df_future = pd.read_csv(file_path + 'future.csv')
df_future['일자'] = pd.to_datetime(df_future['일자'])
df_future_sorted = df_future[['일자', '종가']].sort_values('종가')
df_future_sorted['종가'] = df_future_sorted['종가'].str.replace(',', '').replace('-', np.nan).astype(float)
df_future_sorted.sort_values('일자', inplace=True)
df_future_sorted.set_index('일자', inplace=True)

df_3years = pd.read_csv(file_path + '3years.csv')
df_3years['날짜'] = pd.to_datetime(df_3years['날짜'])
df_3years_sorted = df_3years[['날짜', '값']].sort_values('날짜')
df_3years_sorted.set_index('날짜', inplace=True)

df_nasdaq = pd.read_csv(file_path + 'nasdaq.csv')
df_nasdaq['날짜'] = pd.to_datetime(df_nasdaq['날짜'])
df_nasdaq_sorted = df_nasdaq[['날짜', '값']].sort_values('날짜')
df_nasdaq_sorted['값'] = df_nasdaq_sorted['값'].str.replace(',', '').replace('-', np.nan).astype(float)
df_nasdaq_sorted.sort_values('날짜', inplace=True)
df_nasdaq_sorted.set_index('날짜', inplace=True)

###############################################################################################################################

# 각 종목코드에 대해서 모델 학습 및 추론 반복

for code in tqdm(unique_codes):

    # 학습 데이터 생성
    train_data = train[train['종목코드'] == code][['일자', '종가', '거래량']]
    train_data['일자'] = pd.to_datetime(train_data['일자'], format='%Y%m%d')
    train_data.set_index('일자', inplace=True)
    tc = train_data['종가']
    volume = train_data['거래량']

    # KOSPI 지수 데이터 가져오기
    kospi_data = df_kospi[['Adj Close']].copy()
    kospi_data.rename(columns={'Adj Close': 'KOSPI'}, inplace=True)

    # USD/KRW 환율 데이터 가져오기
    usd_data = df_USD[['Adj Close']].copy()
    usd_data.rename(columns={'Adj Close': 'USD'}, inplace=True)

    # 비트코인 데이터 가져오기
    bitcoin_data = df_bitcoin[['Adj Close']].copy()
    bitcoin_data.rename(columns={'Adj Close': 'Bitcoin'}, inplace=True)

    # OIL 데이터 가져오기
    oil_data = df_oil[['Adj Close']].copy()
    oil_data.rename(columns={'Adj Close': 'Oil'}, inplace=True)

    # GOLD 데이터 가져오기
    gold_data = df_gold[['Adj Close']].copy()
    gold_data.rename(columns={'Adj Close': 'Gold'}, inplace=True)

    # 국채 거래대금 가져오기
    bond_data = df_bond_sorted[['국채_거래대금']].copy()
    bond_data.rename(columns={'국채_거래대금': 'Bond'}, inplace=True)

    # MO 가져오기
    MO_data = df_MO_sorted[['MO']].copy()
    MO_data.rename(columns={'MO': 'MO'}, inplace=True)

    # 금리 데이터 가져오기
    interest_data = df_interest[['Adj Close']].copy()
    interest_data.rename(columns={'Adj Close': 'Interest'}, inplace=True)

    # 선물 데이터 가져오기
    future_data = df_future_sorted[['종가']].copy()
    future_data.rename(columns={'종가': 'Future'}, inplace=True)

    # 국채(3년) 데이터 가져오기
    years_data = df_3years_sorted[['값']].copy()
    years_data.rename(columns={'값': '3Years'}, inplace=True)

    #나스닥 데이터 가져오기
    nasdaq_data = df_nasdaq_sorted[['값']].copy()
    nasdaq_data.rename(columns={'값': 'Nasdaq'}, inplace=True)

    # 종목 데이터와 KOSPI, USD, 비트코인, 거래량, OIL, GOLD 데이터 병합
    merged_data = pd.merge(train_data, kospi_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, usd_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, bitcoin_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, bond_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, oil_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, gold_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, MO_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, interest_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, future_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, years_data, left_index=True, right_index=True, how='outer')
    merged_data = pd.merge(merged_data, nasdaq_data, left_index=True, right_index=True, how='outer')

    # 트레인 데이터의 행을 인덱스로 설정하고 나머지 잘라주기
    merged_data = merged_data.loc[train_data.index]

    # 선형 보간
    merged_data['종가'] = merged_data['종가'].interpolate(method='linear')
    merged_data['거래량'] = merged_data['거래량'].interpolate(method='linear')
    merged_data['KOSPI'] = merged_data['KOSPI'].interpolate(method='linear')
    merged_data['USD'] = merged_data['USD'].interpolate(method='linear')
    merged_data['Bitcoin'] = merged_data['Bitcoin'].interpolate(method='linear')
    merged_data['Oil'] = merged_data['Oil'].interpolate(method='linear')
    merged_data['Gold'] = merged_data['Gold'].interpolate(method='linear')
    merged_data['Bond'] = merged_data['Bond'].interpolate(method='linear')
    merged_data['Bond'] = merged_data['Bond'].interpolate(method='linear')
    merged_data['MO'] = merged_data['MO'].interpolate(method='linear')
    merged_data['Interest'] = merged_data['Interest'].interpolate(method='linear')
    merged_data['Future'] = merged_data['Future'].interpolate(method='linear')
    merged_data['3Years'] = merged_data['3Years'].interpolate(method='linear')
    merged_data['Nasdaq'] = merged_data['Nasdaq'].interpolate(method='linear')

    # 학습 데이터 다시 분리
    tc = merged_data['종가']
    volume = merged_data['거래량']
    kospi = merged_data['KOSPI']
    usd = merged_data['USD']
    bitcoin = merged_data['Bitcoin']
    oil = merged_data['Oil']
    gold = merged_data['Gold']
    bond = merged_data['Bond']
    MO = merged_data['MO']
    interest = merged_data['Interest']
    future = merged_data['Future']
    years =  merged_data['3Years']
    nasdaq = merged_data['Nasdaq']

    valid_indices = ~pd.concat([tc,volume, kospi, usd, bitcoin, oil, gold, bond, MO, interest, future, years, nasdaq], axis=1).isnull().any(axis=1)
    filtered_tc = tc[valid_indices]
    filtered_volume = volume[valid_indices]
    filtered_kospi = kospi[valid_indices]
    filtered_usd = usd[valid_indices]
    filtered_bitcoin = bitcoin[valid_indices]
    filtered_oil = oil[valid_indices]
    filtered_gold = gold[valid_indices]
    filtered_bond = bond[valid_indices]
    filtered_MO = MO[valid_indices]
    filtered_interest = interest[valid_indices]
    filtered_future = future[valid_indices]
    filtered_years = years[valid_indices]
    filtered_nasdaq = nasdaq[valid_indices]

    # 모델 선언, 학습시키기
    model = ARIMA(filtered_tc, exog=pd.concat([filtered_kospi, filtered_usd, filtered_interest, filtered_future, filtered_years, filtered_nasdaq], axis=1), order=(0, 1, 0))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15, exog=pd.concat([filtered_kospi[-15:], filtered_usd[-15:], filtered_interest[-15:], filtered_future[-15:], filtered_years[-15:], filtered_nasdaq[-15:]], axis=1))

    # 예측값 반환
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 오차율 계산
    actual_value = filtered_tc[-1]  # 실제 종가
    predicted_value = predictions.iloc[-1]  # 예측 종가
    squared_error = np.square(predicted_value - actual_value)
    mean_squared_error = np.mean(squared_error)
    rmse = np.sqrt(mean_squared_error)

    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return, 'rmse' : rmse}, ignore_index=True)

results_df['순위'] = results_df['final_return'].rank(method='first').astype('int') # 각 순위를 중복없이 생성

results_df

##############################################################################################################################################################

# 차분 결정

data = merged_data['종가']
n_diffs = ndiffs(data, alpha=0.05, test='adf', max_d=6)
print(f"추정된 차수 d = {n_diffs}") # 결과 : 추정된 차수 d = 1

# 모형 차수 결정
model = pm.auto_arima(
            y=data,
            d=1,
            start_p=0, max_p=3,
            start_q=0, max_q=3,
            m=1, seasonal=False, # 계절성이 없음!
            stepwise=True,
            trace=True
)
print(model_fit.summary()) # Best model:  ARIMA(0,1,0)(0,0,0)[0]

# 잔차 검정

model_fit.plot_diagnostics()
plt.show()

##############################################################################################################################################################

correlation_matrix = merged_data[['KOSPI','USD','Interest','Future', '3Years', 'Nasdaq']].corr()

# VIF 계산
vif = pd.DataFrame()
vif["Variable"] = correlation_matrix.index
vif["VIF"] = [variance_inflation_factor(correlation_matrix.values, i) for i in range(len(correlation_matrix))]

print(vif)
'''
  Variable       VIF
0      거래량  1.087467
1      Oil  1.225519
2     Gold  1.042834
3     Bond  1.243902
4       MO  1.086335
'''
##############################################################################################################################################################

# 결과 저장하기

'''
아이티윌 컴 경로
file_path = 'C:/Users/itwill/Desktop/'
results_df.to_csv(file_path + 'results_df.csv', index=False)
'''
file_path =  '/content/drive/MyDrive/데이콘 주식 공모전 아이티윌/'
results_df.to_csv(file_path + 'results_df.csv', index=False)

results_submission =results_df[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
results_submission.to_csv(file_path + 'results_submission.csv', index=False)


Mounted at /content/drive


FileNotFoundError: ignored

In [ ]:
# -*- coding: utf-8 -*-
##############################################################################################################################################################

# 데이콘 주식 공모전

##############################################################################################################################################################
# 작성자 : 최규진, 심태율 , 김홍표, 유진권
##############################################################################################################################################################

# 종목 코드와 기간 설정(우선 삼성전자 주가로 함(임시))

stock_code = '005930'  # 삼성전자 종목 코드
start_date = '2022-01-01'
end_date = '2022-12-31'

#다중선형회귀를 이용한 선형회귀식 구하기

#(Y변수)

import pandas as pd
import FinanceDataReader as fdr

stock_code = '005930'  # 삼성전자 종목 코드
start_date = '2022-01-01'
end_date = '2022-12-31'

# 삼성전자 종가 데이터 불러오기
df_main = fdr.DataReader(stock_code, start_date, end_date)
df_close = df_main['Close']
print(df_close)  # Length: 246

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_close_full = df_close.reindex(date_range, fill_value=pd.NA)
df_close_full = pd.DataFrame(df_close_full, columns=['Close'])  # Series를 DataFrame으로 변환

print(df_close_full)  # 결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_close_filled = df_close_full.interpolate(method='linear', limit_direction='both')
print(df_close_filled)

#Length: 365

#총 365개의 행으로 수정된 데이터프레임 생성함(결측치 없음)

df_main = fdr.DataReader(stock_code, start_date, end_date)
df_close = df_main['Close']
print(df_close)  # Length: 246

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_close_full = df_close.reindex(date_range, fill_value=pd.NA)
df_close_full = pd.DataFrame(df_close_full, columns=['Close'])  # Series를 DataFrame으로 변환

print(df_close_full)  # 결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_close_filled = df_close_full.interpolate(method='linear', limit_direction='both')
print(df_close_filled)


##############################################################################################################################################################

#(X변수)

#삼성전자 거래량 데이터 불러오기
df_volume = df_main['Volume']
print(df_volume) # Length: 246

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_volume_full = df_volume.reindex(date_range, fill_value=pd.NA)
df_volume_full = pd.DataFrame(df_volume_full, columns=['Volume'])  # Series를 DataFrame으로 변환

print(df_volume_full)  # 결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_volume_filled = df_volume_full.interpolate(method='linear', limit_direction='both')
print(df_volume_filled)

##############################################################################################################################################################

# KOSPI 데이터 불러오기
df_kospi = fdr.DataReader('KS11', start_date, end_date)
print(df_kospi['Close']) # Length: 244

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_kospi_full = df_kospi.reindex(date_range, fill_value=pd.NA)

print(df_kospi_full) #결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_kospi_filled = df_kospi_full.interpolate(method='linear', limit_direction='both')
print(df_kospi_filled)

##############################################################################################################################################################

#원/달러 환율 불러오기
df_USD = fdr.DataReader('USD/KRW', start_date, end_date)
print(df_USD['Close']) # Length: 260

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_USD_full = df_USD.reindex(date_range, fill_value=pd.NA)

print(df_USD_full) #결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_USD_filled = df_USD_full.interpolate(method='linear', limit_direction='both')
print(df_USD_filled)

##############################################################################################################################################################

# 미국 금리 데이터 불러오기
df_interest = fdr.DataReader('DGS', start_date, end_date)
print(df_interest['Close']) # Length: 251

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_interest_full = df_interest.reindex(date_range, fill_value=pd.NA)

print(df_interest_full) #결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_interest_filled = df_interest_full.interpolate(method='linear', limit_direction='both')
print(df_interest_filled)

##############################################################################################################################################################

# KOSPI 외국인 순매수량 데이터 불러오기
df_foreign = fdr.DataReader('KS11', start_date, end_date)
print(df_foreign['Close']) # Length: 244

# 전체 기간의 날짜 목록 생성
date_range = pd.date_range(start=start_date, end=end_date)

# 빈 날짜 행을 추가하여 데이터프레임 완성
df_foreign_full = df_foreign.reindex(date_range, fill_value=pd.NA)

print(df_foreign_full) #결측값이 아직 존재함

# 결측치를 가장 가까운 값으로 선형 보간하여 채움
df_foreign_filled = df_foreign_full.interpolate(method='linear', limit_direction='both')
print(df_foreign_filled)

##############################################################################################################################################################

#할일 1: 데이터프레임 Length 맞추기

# 데이터프레임 생성
df = pd.DataFrame({'Close':df_close_filled['Close'], 'Volume': df_volume_filled['Volume'],
                   'KOSPI': df_kospi_filled['Close'], 'InterestRate': df_interest_filled['Close'],
                   'ForeignBuy': df_foreign_filled['Close'], 'USD': df_USD_filled['Close']})

print(df)



# 종속변수와 독립변수 분리
X = df[['Volume', 'KOSPI', 'InterestRate', 'ForeignBuy','USD']]  # 독립변수
y = df['Close']  # 종속변수

def get_scores_f(model, xtrain, xtest, ytrain, ytest):
    pred1 = model.predict(xtrain)
    pred2 = model.predict(xtest)


    A1 = r2_score(ytrain, pred1)
    A2 = r2_score(ytest, pred2)
    pred2 = np.where(pred2<0 , 0 , pred2)
    B = mse(ytest, pred2)
    C = B ** 0.5

    data = [round(x,4) for x in [A1, A2, C]]
    names = 'r2_train r2_test rmse'.split()
    scores = pd.Series(data, index = names)
    return scores

def make_models_f(xtrain, xtest, ytrain, ytest, n=500):
    temp = pd.DataFrame()

    model = RandomForestRegressor(n, random_state =0).fit(xtrain, ytrain)
    temp['model'] = get_scores_f(model, xtrain, xtest, ytrain, ytest)

    for d in range(3,9):
        model = RandomForestRegressor(n, max_depth=d, random_state = 0).fit(xtrain, ytrain)
        temp[f'model_{d}'] = get_scores_f(model, xtrain, xtest, ytrain, ytest)

    temp = temp.T
    temp.insert(2, 'diff', (temp['r2_train'] - temp['r2_test']).abs())

    return temp




xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)


model = make_models_f(xtrain, xtest, ytrain,ytest)
print(model)



In [ ]:
temp = pd.DataFrame(model)

temp.head()






,r2_train,r2_test,diff,rmse
model,0.9972,0.9819,0.0153,903.7856
model_3,0.9671,0.9661,0.0010,1237.7349
model_4,0.9804,0.9727,0.0077,1109.9579
model_5,0.9907,0.9791,0.0116,971.6831
model_6,0.9945,0.9812,0.0133,922.3535


In [ ]:
#샤프 계수 관련 코드

file_path = 'C:/Users/itwill/Desktop/train.csv'
df = pd.read_csv(file_path)
df.info()

df.head(20)

df_grouped = df.groupby('종목코드').apply(lambda x: x.iloc[:, [1, 0, 2, 3, 4, 5, 6, 7]]).reset_index(drop=True)

# 각 그룹 내에서 날짜를 오름차순으로 정렬
df_sorted = df_grouped.groupby('종목코드').apply(lambda x: x.sort_values('일자')).reset_index(drop=True)

# 결과 출력
print(df_sorted)


# 사용자 입력 날짜
date1 = input("첫 번째 날짜를 입력하세요 (yyyymmdd): ")
date2 = input("두 번째 날짜를 입력하세요 (yyyymmdd): ")

# 선택한 날짜의 인덱스 설정
date1_index = df[df['일자'] == int(date1)].index[0]
date2_index = df[df['일자'] == int(date2)].index[0]

# "최종 매수 수익률" 계산
final_profit = (df.loc[date2_index, '종가'] - df.loc[date1_index, '종가']) / df.loc[date1_index, '종가']

# 결과 출력
print("최종 매수 수익률:", final_profit)


import pandas as pd
import numpy as np

# 주식 수익률 데이터
returns = pd.Series([0.01, 0.02, 0.03, 0.04, 0.01, 0.02, -0.01, -0.02, 0.01, 0.03])

# 무위험 수익률
risk_free_rate = 0.035

# 일간 수익률 변동성
volatility = returns.std()

# 최종 매수 수익률
final_buy_profit = (returns.iloc[-1] - returns.iloc[0]) / returns.iloc[0]

# 최종 공매도 수익률 총합
total_short_profit = (returns.iloc[-1] - returns.iloc[0]) / returns.iloc[0] * (-1)

# 총 자산 최종 수익률
total_returns = (final_buy_profit + total_short_profit) / 400

# 매매일수
trading_days = len(returns)

# 연율화된 총 자산 수익률
annual_returns = returns.sum() * 250

# 일간 수익률 변동성
daily_volatility = np.sqrt(((returns[1:] - returns.shift(1)) / returns.shift(1)) ** 2).mean() * np.sqrt(250)

# 일간 공매도 수익률
daily_short_profit = (returns.iloc[-1] - returns.iloc[0]) / returns.iloc[0] * (-1)

# 샤프지수 계산
sharpe_ratio = (annual_returns - risk_free_rate) / daily_volatility

# 결과 출력
print("일간 수익률 변동성 (Volatility):", volatility)
print("연율화된 총 자산 수익률 (Annual Returns):", annual_returns)
print("총 자산 일간 수익률 변동성 (Daily Volatility):", daily_volatility)
print("샤프지수 (Sharpe Ratio):", sharpe_ratio)
print("매매일수:", trading_days)

'''
# 전날 종가 데이터를 가져오기 위해 '종가' 열을 shift하여 새로운 '전날 종가' 열 생성
df['전날 종가'] = df['종가'].shift(1)

# '일간 공매도 수익률' 계산
df['일간 공매도 수익률'] = (df['종가'] - df['전날 종가']) / df['전날 종가'] * -1

# 결과 출력
df.info()

# '일간 매수 수익률' 계산
df['일간 매수 수익률'] = (df['종가'] - df['전날 종가']) / df['전날 종가']

df['연율화된 일간 수익률'] = df['일간 매수 수익률'] * 250

# 결과 출력
print(df)

# '일간 수익률'의 평균 계산
mean_daily_return = df['일간 매수 수익률'].mean()

# '총자산 일간 수익률 변동성' 계산
n = 15  # 변동성을 계산할 일수
df['총자산 일간 수익률 변동성'] = np.sqrt(((df['연율화된 일간 수익률'] - df['일간 매수 수익률'].mean()) ** 2).rolling(n-2).mean())

# 결과 출력
print(df)
df.info()
'''